In [10]:
# Import dependencies
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.ensemble import RandomForestClassifier
import google.colab.auth
from google.colab import drive
import os

In [11]:
# Read in spreadsheet
from google.colab import drive
import os
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [12]:
# Read CSV file directly without changing directory
file_path = '/content/drive/My Drive/Colab Notebooks/Resources/diabetes_2_classes.csv'
diabetes_df = pd.read_csv(file_path)
diabetes_df.head()

,Diabetes_binary,HighBP,HighChol,CholCheck,BMI,Smoker,Stroke,HeartDiseaseorAttack,PhysActivity,Fruits,...,AnyHealthcare,NoDocbcCost,GenHlth,MentHlth,PhysHlth,DiffWalk,Sex,Age,Education,Income
0,0.0,1.0,1.0,1.0,40.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,5.0,18.0,15.0,1.0,0.0,9.0,4.0,3.0
1,0.0,0.0,0.0,0.0,25.0,1.0,0.0,0.0,1.0,0.0,...,0.0,1.0,3.0,0.0,0.0,0.0,0.0,7.0,6.0,1.0
2,0.0,1.0,1.0,1.0,28.0,0.0,0.0,0.0,0.0,1.0,...,1.0,1.0,5.0,30.0,30.0,1.0,0.0,9.0,4.0,8.0
3,0.0,1.0,0.0,1.0,27.0,0.0,0.0,0.0,1.0,1.0,...,1.0,0.0,2.0,0.0,0.0,0.0,0.0,11.0,3.0,6.0
4,0.0,1.0,1.0,1.0,24.0,0.0,0.0,0.0,1.0,1.0,...,1.0,0.0,2.0,3.0,0.0,0.0,0.0,11.0,5.0,4.0


In [13]:
#FEATURE ENGINEERING
diabetes_df["Age_/_BMI"] = diabetes_df["BMI"] / diabetes_df["Age"]
diabetes_df["MentHlth_*_BMI"] = diabetes_df["MentHlth"] * diabetes_df["BMI"]
diabetes_df["Income_/_BMI"] = diabetes_df["Income"] / diabetes_df["BMI"]
diabetes_df["PhysHlth_/MentHlth"] = diabetes_df["PhysHlth"] / diabetes_df["MentHlth"]
diabetes_df.head(100)

,Diabetes_binary,HighBP,HighChol,CholCheck,BMI,Smoker,Stroke,HeartDiseaseorAttack,PhysActivity,Fruits,...,MentHlth,PhysHlth,DiffWalk,Sex,Age,Education,Income,Age_/_BMI,MentHlth_*_BMI,Income_/_BMI
0,0.0,1.0,1.0,1.0,40.0,1.0,0.0,0.0,0.0,0.0,...,18.0,15.0,1.0,0.0,9.0,4.0,3.0,4.444444,720.0,0.075000
1,0.0,0.0,0.0,0.0,25.0,1.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,7.0,6.0,1.0,3.571429,0.0,0.040000
2,0.0,1.0,1.0,1.0,28.0,0.0,0.0,0.0,0.0,1.0,...,30.0,30.0,1.0,0.0,9.0,4.0,8.0,3.111111,840.0,0.285714
3,0.0,1.0,0.0,1.0,27.0,0.0,0.0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,11.0,3.0,6.0,2.454545,0.0,0.222222
4,0.0,1.0,1.0,1.0,24.0,0.0,0.0,0.0,1.0,1.0,...,3.0,0.0,0.0,0.0,11.0,5.0,4.0,2.181818,72.0,0.166667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,1.0,1.0,1.0,1.0,25.0,1.0,0.0,1.0,0.0,1.0,...,15.0,30.0,1.0,0.0,9.0,2.0,3.0,2.777778,375.0,0.120000
96,1.0,0.0,0.0,1.0,32.0,0.0,0.0,0.0,1.0,0.0,...,0.0,3.0,0.0,0.0,3.0,5.0,3.0,10.666667,0.0,0.093750
97,0.0,1.0,0.0,1.0,45.0,0.0,0.0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,9.0,4.0,6.0,5.000000,0.0,0.133333
98,0.0,1.0,1.0,1.0,28.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,1.0,0.0,11.0,4.0,3.0,2.545455,0.0,0.107143


# Logistic Regression Analysis

In [14]:
# Set our target and feature variables for the ML model
y = diabetes_df['Diabetes_binary']
X = diabetes_df.drop(columns='Diabetes_binary')

In [15]:
# Use SKlearn to train the data
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

## Need to scale only numerical data here

In [16]:
# Create the StandardScaler instance
scaler = StandardScaler()

# Fit the Standard Scaler with the training data
X_scaler = scaler.fit(X_train)

# Scale the training data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [17]:
# Create a logistic regression model
classifier = LogisticRegression(solver='lbfgs', random_state=1)

# Define the logistic regression model
log_classifier = LogisticRegression(solver="lbfgs",max_iter=500)

# Train the model
log_classifier.fit(X_train,y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(max_iter=500)

In [18]:
# Score the model
print(f"Training Data Score: {log_classifier.score(X_train, y_train)}")
print(f"Testing Data Score: {log_classifier.score(X_test, y_test)}")

Training Data Score: 0.8636024387680017
Testing Data Score: 0.8621570482497635


In [19]:
# Predict outcomes for test data set
predictions = log_classifier.predict(X_test)
pd.DataFrame({"Prediction": predictions, "Actual": y_test})

,Prediction,Actual
235899,0.0,0.0
74852,0.0,1.0
8205,0.0,0.0
127632,0.0,1.0
32021,0.0,0.0
...,...,...
108360,0.0,0.0
8531,0.0,0.0
183429,0.0,0.0
52932,0.0,1.0


In [20]:
# Create and save the training classification report
training_report = classification_report(y_test, predictions)

# Print the training classification report
print(training_report)

              precision    recall  f1-score   support

         0.0       0.88      0.98      0.92     54551
         1.0       0.53      0.14      0.22      8869

    accuracy                           0.86     63420
   macro avg       0.70      0.56      0.57     63420
weighted avg       0.83      0.86      0.83     63420



In [21]:
# Accuracy score for logistic regression
y_pred = log_classifier.predict(X_test)
print(f" Logistic regression model accuracy: {accuracy_score(y_test,y_pred):.3f}")

 Logistic regression model accuracy: 0.862


# Random Forest Classification

In [22]:
# Create the random forest classifier instance
rf_model = RandomForestClassifier(n_estimators=500, random_state=78)

In [23]:
# Fit the model and use .ravel()on the "y_train" data.
rf_model = rf_model.fit(X_train_scaled, y_train.ravel())

In [24]:
# Making predictions using the testing data
rf_predictions = rf_model.predict(X_test_scaled)

In [25]:
# Get the feature importance array
importances = rf_model.feature_importances_
# List the top 10 most important features
importances_sorted = sorted(zip(rf_model.feature_importances_, X.columns), reverse=True)
importances_sorted

[(0.11157440308932401, 'Age_/_BMI'),
 (0.11124999565323607, 'Income_/_BMI'),
 (0.09554136194954987, 'BMI'),
 (0.06844495097753754, 'PhysHlth'),
 (0.06753766878898965, 'Age'),
 (0.06678777412280439, 'GenHlth'),
 (0.06140947927721629, 'Education'),
 (0.048120762416766764, 'Income'),
 (0.0474777451249671, 'MentHlth_*_BMI'),
 (0.04094592308869879, 'HighBP'),
 (0.03484920593010642, 'MentHlth'),
 (0.031350879903565276, 'Smoker'),
 (0.03099155725211924, 'Fruits'),
 (0.026892203216015882, 'HighChol'),
 (0.02635209385968902, 'Sex'),
 (0.024596017104237363, 'PhysActivity'),
 (0.02393033702219971, 'Veggies'),
 (0.02264492576035539, 'DiffWalk'),
 (0.018356819814224275, 'HeartDiseaseorAttack'),
 (0.012307299534393476, 'NoDocbcCost'),
 (0.011055072742135985, 'Stroke'),
 (0.0072465055224427794, 'HvyAlcoholConsump'),
 (0.007153001246108518, 'AnyHealthcare'),
 (0.0031840166033162143, 'CholCheck')]

In [26]:
# Create and save the training classification report
rf_training_report = classification_report(y_test, rf_predictions)

# Print the training classification report
print(rf_training_report)

              precision    recall  f1-score   support

         0.0       0.88      0.97      0.92     54551
         1.0       0.50      0.17      0.25      8869

    accuracy                           0.86     63420
   macro avg       0.69      0.57      0.59     63420
weighted avg       0.82      0.86      0.83     63420



In [27]:
print(training_report)

              precision    recall  f1-score   support

         0.0       0.88      0.98      0.92     54551
         1.0       0.53      0.14      0.22      8869

    accuracy                           0.86     63420
   macro avg       0.70      0.56      0.57     63420
weighted avg       0.83      0.86      0.83     63420



In [28]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test, rf_model.predict_proba(X_test_scaled)[:, 1])

0.7999211467772208

TAKING AWAY CERTAIN COLUMNS THAT HAVE LOW IMPORTANCE

In [33]:
less_columns = ["Diabetes_binary", "CholCheck", "HvyAlcoholConsump", "AnyHealthcare", "Stroke", "NoDocbcCost", "HeartDiseaseorAttack", "DiffWalk", "PhysActivity", "Veggies", "HighChol", "Sex"]
y = diabetes_df["Diabetes_binary"]
X = diabetes_df.drop(columns=less_columns)

In [34]:
# Use SKlearn to train the less data
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [35]:
# Create the StandardScaler instance
scaler = StandardScaler()

# Fit the Standard Scaler with the training data
X_scaler = scaler.fit(X_train)

# Scale the training data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [36]:
# Create a logistic regression model
classifier = LogisticRegression(solver='lbfgs', random_state=1)

# Define the logistic regression model
log_classifier = LogisticRegression(solver="lbfgs",max_iter=500)

# Train the model
log_classifier.fit(X_train,y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(max_iter=500)

In [37]:
# Score the model
print(f"Training Data Score: {log_classifier.score(X_train, y_train)}")
print(f"Testing Data Score: {log_classifier.score(X_test, y_test)}")

Training Data Score: 0.8626248291811206
Testing Data Score: 0.8615736360769474


In [38]:
# Predict outcomes for test data set
predictions = log_classifier.predict(X_test)
pd.DataFrame({"Prediction": predictions, "Actual": y_test})

,Prediction,Actual
235899,0.0,0.0
74852,0.0,1.0
8205,0.0,0.0
127632,1.0,1.0
32021,0.0,0.0
...,...,...
108360,0.0,0.0
8531,0.0,0.0
183429,0.0,0.0
52932,0.0,1.0


In [39]:
# Create and save the training classification report
training_report = classification_report(y_test, predictions)

# Print the training classification report
print(training_report)

              precision    recall  f1-score   support

         0.0       0.87      0.98      0.92     54551
         1.0       0.52      0.12      0.20      8869

    accuracy                           0.86     63420
   macro avg       0.70      0.55      0.56     63420
weighted avg       0.82      0.86      0.82     63420



In [40]:
# Accuracy score for logistic regression
y_pred = log_classifier.predict(X_test)
print(f" Logistic regression model accuracy: {accuracy_score(y_test,y_pred):.3f}")

 Logistic regression model accuracy: 0.862


RANDOM FOREST CLASSIFICATION ON NEW DATA

In [41]:
# Create the random forest classifier instance
rf_model = RandomForestClassifier(n_estimators=500, random_state=78)

In [42]:
# Fit the model and use .ravel()on the "y_train" data.
rf_model = rf_model.fit(X_train_scaled, y_train.ravel())

In [43]:
# Making predictions using the testing data
rf_predictions = rf_model.predict(X_test_scaled)

In [44]:
# Get the feature importance array
importances = rf_model.feature_importances_
# List the top 10 most important features
importances_sorted = sorted(zip(rf_model.feature_importances_, X.columns), reverse=True)
importances_sorted

[(0.12341035009241431, 'Income_/_BMI'),
 (0.12092601785265192, 'Age_/_BMI'),
 (0.1086998054093029, 'PhysHlth'),
 (0.09846832865471696, 'BMI'),
 (0.09509593776916028, 'Education'),
 (0.08362699945683473, 'GenHlth'),
 (0.07218792154034023, 'Age'),
 (0.062435492429506946, 'MentHlth_*_BMI'),
 (0.05089334301224428, 'HighBP'),
 (0.04953091774344729, 'Income'),
 (0.04523000764180195, 'Smoker'),
 (0.045115021415100286, 'Fruits'),
 (0.044379856982477824, 'MentHlth')]

In [45]:
# Create and save the training classification report
rf_training_report = classification_report(y_test, rf_predictions)

# Print the training classification report
print(rf_training_report)

              precision    recall  f1-score   support

         0.0       0.88      0.95      0.92     54551
         1.0       0.41      0.19      0.26      8869

    accuracy                           0.85     63420
   macro avg       0.64      0.57      0.59     63420
weighted avg       0.81      0.85      0.82     63420



In [46]:
print(training_report)

              precision    recall  f1-score   support

         0.0       0.87      0.98      0.92     54551
         1.0       0.52      0.12      0.20      8869

    accuracy                           0.86     63420
   macro avg       0.70      0.55      0.56     63420
weighted avg       0.82      0.86      0.82     63420



In [47]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test, rf_model.predict_proba(X_test_scaled)[:, 1])

0.7658468377622711